<a href="https://colab.research.google.com/github/woo2201/ColabDrive/blob/reservior/comment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/dataset/jigsaw-toxic-severity-rating/

/content/gdrive/MyDrive/dataset/jigsaw-toxic-severity-rating


In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import os
import glob

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
path = "/content/gdrive/MyDrive/dataset/jigsaw-toxic-severity-rating/"
filenames = glob.glob(path + "*.csv")
data = [var.split('.')[0].split('/')[-1] for var in filenames]

In [ ]:
# dynamically allocate df names.
for name in zip(filenames, data):
    loc = list(name)[0]
    val = list(name)[1]
    globals()[val] = pd.read_csv(loc)

In [ ]:
data

['train',
 'test',
 'test_labels',
 'comments_to_score',
 'sample_submission',
 'validation_data']

데이터를 쭉 불러온다 ~ !

# EDA 및 데이터 체크


In [ ]:
comments_to_score.head(3)

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...


In [ ]:
validation_data.head(3)

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."


In [ ]:
train.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
69275,b957bf27225c89d1,That's my point. If they can't make any of th...,0,0,0,0,0,0
142693,fb3c0706595f712a,"""\nI think Sanger is saying that Jimbo's behav...",0,0,0,0,0,0
95333,fee331bcdd154c74,"Unertow confusion \n\nHi Morven, can you expla...",0,0,0,0,0,0
38208,65f9a17d5106f26f,the one's to provoke the reaction,0,0,0,0,0,0
76656,cd45bc054715b7d2,REDIRECT User talk:John Vandenberg/Archive 7,0,0,0,0,0,0


In [ ]:
train['severe_toxic'] = train.severe_toxic * 1.5
train['insult'] = train.insult * 0.1
train['identity_hate'] = train.identity_hate * 1.5
train['toxic'] = train.toxic * 1
train['threat'] = train.threat * 1.5
train['obscene'] = train.obscene * 0.16

train['y'] = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].mean(axis=1)

In [ ]:
# Remove All Columns Leaving Comment Tex and Y
train = train[['comment_text', 'y']].rename(columns={'comment_text': 'text'})

In [ ]:
# Reduce Rows with No Toxicity as our key focus area is the reviews with toxicity
train = pd.concat([train[train.y>0] , 
                   train[train.y==0].sample(int(len(train[train.y>0])*1.5)) ], axis=0).sample(frac=1)

In [ ]:
train.shape

(40562, 2)

In [ ]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5))),
        ("clf", Ridge()),
    ]
)

In [ ]:
pipeline.fit(train['text'], train['y'])

Pipeline(steps=[('vect',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3,
                                 ngram_range=(3, 5))),
                ('clf', Ridge())])

In [ ]:
p1 = pipeline.predict(validation_data['less_toxic'])
p2 = pipeline.predict(validation_data['more_toxic'])

In [ ]:
f'Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}'

'Validation Accuracy is 67.88'

In [ ]:
validation_data

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"
...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook


In [ ]:
lst = [1,2]
for i in range(2, 100000):
    val = (lst[i-2] + lst[i-1])
    if val >= 4000000:
        break    
    lst.append(val)

In [ ]:
sum([i for i in lst if i % 2 == 0])

4613732